# Oczyszczanie danych, przypomnienie i kontynuacja

Podczas analizowania i modelowania danych znaczną część czasu poświęca się na przygotowanie danych: 
1) ładowanie, 
2) czyszczenie, 
3) przekształcanie i 
4) przekładanie. 

Wykonywanie tego typu zadań zajmuje nawet 80% czasu pracy analityka. Na szczęście pakiet pandas oraz elementy wbudowane w Pythona tworzą zestaw uniwersalnych i szybkich wysokopoziomowych narzędzi, które umożliwiają przekształcenie danych do właściwej formy.

Większość elementów pakietu pandas została zaprojektowana i zaimplementowana z myślą o potrzebie rozwiązywania realnych problemów.

## Obsługa brakujących danych
Sposób reprezentacji brakujących danych zaimplementowany w pakiecie pandas można uznać za nieidealny, ale z punktu widzenia wielu użytkowników jest on praktyczny. W przypadku danych numerycznych pakiet pandas korzysta ze zmiennoprzecinkowej wartości NaN (nie-liczba) w celu oznaczenia brakujących danych. Jest to tzw. wartość zastępcza, której występowanie można łatwo wykryć:

In [3]:
import pandas as pd
import numpy as np

float_data = pd.Series([1.2, -3.5, np.nan, 0])
float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

Metoda isna zwraca serię wartości logicznych, w której True oznacza brak wartości w oryginalnej serii:

In [4]:
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

W pakiecie pandas brakujące wartości określamy mianem wartości NA (wartości niedostępnych, z ang. not available). W zastosowaniach statystycznych dane niedostępne mogą być danymi, które nie istnieją, lub danymi, które istnieją, ale nie zostały zaobserwowane (np. z powodu problemu wynikającego ze sposobu zbierania danych). Podczas oczyszczania danych przed przeprowadzeniem ich analizy często warto przeprowadzić analizę samych brakujących wartości. Pozwoli to zidentyfikować problemy wynikające z techniki zbierania danych lub potencjalne tendencje spowodowane brakiem pewnych wartości.
W tablicach za wartość NA przyjmuje się wbudowaną w standardową bibliotekę Pythona wartość None:

In [5]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])
string_data

0    aardvark
1         NaN
2        None
3     avocado
dtype: object

In [6]:
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

In [7]:
float_data = pd.Series([1, 2, None], dtype='float64')
float_data


0    1.0
1    2.0
2    NaN
dtype: float64

In [8]:
float_data.isna()

0    False
1    False
2     True
dtype: bool

Metody przydatne podczas obsługi brakujących danych:

dropna - Filtruje etykiety osi na podstawie występowania brakujących danych; możliwe jest zdefiniowanie zmiennych wartości progowych określających liczbę tolerowanych brakujących danych.


fillna - Wypełnia brakujące dane jakimiś wartościami lub robi to za pomocą metody interpolacji, takiej jak np. "ffill" lub "bfill".

isna - Zwraca wartości logiczne określające miejsce występowania brakujących wartości. 

notna - Negacja isna. Zwraca True, jeżeli wartość istnieje, a False w przeciwnym wypadku.

### Filtrowanie brakujących danych

Istnieje kilka technik umożliwiających odfiltrowanie brakujących danych. Można to zrobić ręcznie za pomocą funkcji pandas.isna i indeksowania wartości logicznych, ale łatwiej jest skorzystać z funkcji dropna. W przypadku obiektu typu Series funkcja ta zwraca tylko dane o wartości innej niż null i indeksy odwołujące się jedynie do tych danych:


In [9]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

Jest to odpowiednik następującego kodu:

In [10]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

Sprawy nieco komplikują się podczas pracy z obiektami DataFrame. Możesz chcieć odrzucić wiersze lub kolumny, które zawierają tylko wartości NA, lub takie, które zawierają chociażby jedną wartość NA. Funkcja dropna domyślnie odrzuca wiersze zawierające przynajmniej jedną brakującą wartość:

In [11]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan], [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [12]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


Przekazanie parametru how='all' spowoduje odrzucenie tylko wierszy, które zawierają same wartości NA:

In [13]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


Pamiętajmy, że opisane funkcje domyślnie zwracają nowe obiekty, tj. nie modyfikują zawartości oryginalnych obiektów.
Aby odrzucić w ten sam sposób kolumny, należy skorzystać z parametru axis="columns":

In [14]:
data[4] = np.nan
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [15]:
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Załóżmy, że chcesz zachować tylko wiersze zawierające określoną liczbę obserwacji. Możesz to zrobić za pomocą argumentu thresh:

In [16]:
df = pd.DataFrame(np.random.randn(7, 3)) 
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df

,0,1,2
0,0.330763,NaN,NaN
1,-0.302005,NaN,NaN
2,0.587787,NaN,1.795828
3,-0.155761,NaN,0.693566
4,0.664223,0.311102,0.295327
5,2.776345,-0.519109,-0.095640
6,0.569251,0.608132,-1.259057


In [17]:
df.dropna()

,0,1,2
4,0.664223,0.311102,0.295327
5,2.776345,-0.519109,-0.095640
6,0.569251,0.608132,-1.259057


In [18]:
df.dropna(thresh=2)

,0,1,2
2,0.587787,NaN,1.795828
3,-0.155761,NaN,0.693566
4,0.664223,0.311102,0.295327
5,2.776345,-0.519109,-0.095640
6,0.569251,0.608132,-1.259057


### Wypełnianie brakujących danych
Zamiast filtrować brakujące dane (i narażać się na jednoczesne usunięcie również innych danych), czasami lepiej jest w jakiś sposób te „dziury” wypełnić. Najczęściej robi się to za pomocą funkcji fillna. Wywołanie funkcji fillna wraz ze stałą spowoduje wstawienie zadeklarowanej wartości w miejsce brakujących danych:

In [19]:
df.fillna(0)

,0,1,2
0,0.330763,0.000000,0.000000
1,-0.302005,0.000000,0.000000
2,0.587787,0.000000,1.795828
3,-0.155761,0.000000,0.693566
4,0.664223,0.311102,0.295327
5,2.776345,-0.519109,-0.095640
6,0.569251,0.608132,-1.259057


Wywołując fillna wraz ze słownikiem, możesz wypełnić brakujące elementy poszczególnych kolumn różnymi wartościami:


In [20]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,0.330763,0.500000,0.000000
1,-0.302005,0.500000,0.000000
2,0.587787,0.500000,1.795828
3,-0.155761,0.500000,0.693566
4,0.664223,0.311102,0.295327
5,2.776345,-0.519109,-0.095640
6,0.569251,0.608132,-1.259057


Metody interpolacji dostępne podczas wykonywania operacji uaktualniania indeksu mogą być również użyte wraz z funkcją ffill (dawniej fillna):

In [21]:
df = pd.DataFrame(np.random.standard_normal((6, 3))) 
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

,0,1,2
0,0.188392,-1.168195,0.561419
1,0.961618,1.900135,0.384788
2,0.138289,NaN,0.025217
3,1.376285,NaN,1.358173
4,-0.206256,NaN,NaN
5,0.121451,NaN,NaN


In [22]:
df.ffill()

,0,1,2
0,0.188392,-1.168195,0.561419
1,0.961618,1.900135,0.384788
2,0.138289,1.900135,0.025217
3,1.376285,1.900135,1.358173
4,-0.206256,1.900135,1.358173
5,0.121451,1.900135,1.358173


In [23]:
df.ffill(limit=2)

,0,1,2
0,0.188392,-1.168195,0.561419
1,0.961618,1.900135,0.384788
2,0.138289,1.900135,0.025217
3,1.376285,1.900135,1.358173
4,-0.206256,NaN,1.358173
5,0.121451,NaN,1.358173


Funkcja fillna wykonuje inne operacje, na przykład interpoluje dane przy użyciu mediany lub średniej:

In [24]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## Przekształcanie danych

Filtrowanie, czyszczenie i inne przekształcenia tworzą kolejną klasę ważnych operacji.

### Usuwanie duplikatów
Istnieje wiele powodów pojawiania się zduplikowanych wierszy w ramkach danych. Oto przy- kład takiej sytuacji:


In [25]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'], 'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4




Metoda duplicated ramki danych zwraca serię wartości logicznych określającą, czy poszczególne wiersze ramki danych są duplikatami, tj. czy wartości w kolejnych kolumnach są identyczne z zawartymi w poprzednim wierszu:


In [26]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Metoda drop_duplicates zwraca ramkę danych, w przypadku której metoda duplicated zwraca same wartości False:


In [27]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Obie z zaprezentowanych metod biorą pod uwagę wszystkie kolumny, ale można wybrać podzbiór kolumn, które mają zostać wzięte pod uwagę podczas wykrywania duplikatów. Załóżmy, że mamy dodatkową kolumnę wartości i chcemy dokonać operacji filtrowania duplikatów tylko na podstawie kolumny k1:


In [28]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [29]:
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


Metody duplicated i drop_duplicates domyślnie zachowują pierwszą znalezioną kombinację wartości. Parametr keep='last' spowoduje zwrócenie ostatniej takiej kombinacji:

In [30]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Przekształcanie danych przy użyciu funkcji lub mapowania

W przypadku wielu zbiorów danych może zachodzić konieczność wykonania pewnych przekształceń na podstawie wartości umieszczonych w tablicy, serii lub kolumnie ramki danych. Zobaczny poniższy hipotetyczn zbiór danych opisujący różne rodzaje mięsa:

In [31]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 'Bacon', 'pastrami', 'honey ham', 'nova lox'], 'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Załóżmy, że chcemy dodać kolumnę określającą zwierzę, z którego zrobiony jest każdy z produktów spożywczych. Utwórzmy mapowanie każdego typu mięsa do zwierzęcia:

In [32]:
meat_to_animal = { 'bacon': 'pig', 'pulled pork': 'pig', 'pastrami': 'cow', 'corned beef': 'cow', 'honey ham': 'pig', 'nova lox': 'salmon'
}

Metoda map używana na obiektach Series przyjmuje funkcję lub słownik — element definiujący mapowanie:


In [33]:
data['animal'] = data['food'].str.lower().map(meat_to_animal)

In [34]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


Istnieje również możliwość przekazania funkcji wykonującej wszystkie operacje:

In [35]:
def get_animal(x):
    return meat_to_animal[x]

In [36]:
data['food'].str.lower().map(get_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Zastępowanie wartości
Wypełnianie brakujących danych za pomocą metody fillna to specjalny przypadek zastosowania bardziej uniwersalnego mechanizmu zastępowania wartości. 
 
Oto przykładowy obiekt typu Series:

In [37]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

Wartości –999 mogą zastępować brakujące dane. W celu zastąpienia ich wartością NaN możemy skorzystać z funkcji replace, która wygeneruje nowy obiekt typu Series:

In [38]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

Jeżeli chcemy za jednym zamachem zastąpić wiele wartości, możemy przekazać do tej funkcji listę wartości, które mają zostać zastąpione:

In [39]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

W celu zastąpienia każdej z wartości inną należy przekazać listę elementów zastępujących:

In [40]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

Przekazany argument może być również słownikiem:

In [41]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

## Zmiana nazw indeksów osi
Etykiety osi mogą być przekształcone podobnie jak wartości obiektu Series. W celu uzyskania nowych obiektów o innych etykietach można skorzystać z funkcji lub mapowania. Osie mogą być również modyfikowane w miejscu - bez tworzenia nowych struktur danych. Oto prosty przykład:


In [42]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
index=['Ohio', 'Colorado', 'New York'], columns=['one', 'two', 'three', 'four'])

Indeksy osi, podobnie jak obiekty Series, obsługują metodę map:

In [43]:
def transform(x):
    return x[:4].upper()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

Przypisywanie do obiektu index może modyfikować ramkę danych w miejscu:

In [44]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


Jeżeli chcemy utworzyć przekształconą wersję zbioru danych bez modyfikowania oryginalnego zbioru, to warto skorzystać z metody rename:


In [45]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


Metoda rename może być używana w połączeniu z obiektem będącym słownikiem zawierającym nowe wartości podzbioru etykiet osi: 

In [46]:
data.rename(index={'OHIO': 'INDIANA'}, columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


Korzystanie z metody rename zwalnia z obowiązku ręcznego kopiowania ramki danych i przypisywa- nia jej atrybutów index i columns.

## Dyskretyzacja i podział na koszyki
Ciągłe dane są często poddawane dyskretyzacji lub dzielone w inny sposób na „koszyki” umożliwiające ich dalszą analizę. Załóżmy, że dysponujesz danymi dotyczącymi osób wchodzących w skład grupy badawczej i chcesz dokonać ich podziału na koszyki w zależności od ich wieku:

In [47]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Dokonajmy podziału na koszyki 18 – 25, 26 – 36, 36 – 60 i 61+. W tym celu możemy skorzystać z funkcji pandas.cut:


In [48]:
bins = [18, 25, 35, 60, 100]
age_categories = pd.cut(ages, bins)
age_categories

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

Obiekt zwrócony przez pandas jest specjalnym obiektem kategorycznym (typu Categorical). Zapre- zentowane dane wyjściowe opisują koszyki wygenerowane za pomocą funkcji pandas.cut. Każdy koszyk jest identyfikowany przez specjalny, charakterystyczny dla biblioteki pandas interwał, opisujący dolną i górną granicę:

In [49]:
age_categories.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [50]:
age_categories.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [51]:
age_categories.categories[0]

Interval(18, 25, closed='right')

In [52]:
age_categories

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [53]:
age_categories.value_counts()

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
Name: count, dtype: int64

Zauważ, że polecenie age_categories.value_counts() zwraca liczby elementów znajdujących się w poszczególnych koszykach będących wynikiem funkcji pandas.cut.
Zgodnie z matematycznym zapisem interwałów nawias okrągły oznacza otwarcie zbioru, a nawias kwadratowy jego domknięcie (wartość graniczna zalicza się do zbioru). Zmiany otwartej strony zbioru można dokonać za pomocą argumentu right=False:

In [54]:
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

Można zdefiniować własne nazwy koszyków. Wystarczy przekazać nazwę listy lub tablicę nazw za pomocą opcji labels:

In [55]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

Jeżeli funkcji pandas.cut przekażemy liczbę całkowitą koszyków zamiast jawnych definicji krawędzi koszyków, to polecenie to zwróci koszyki o równej długości na podstawie minimalnej i maksymalnej wartości znajdującej się w dzielonym zbiorze danych. Oto przykład danych o równym rozkładzie podzielonych na cztery koszyki:

In [56]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.05, 0.27], (0.5, 0.72], (0.5, 0.72], (0.5, 0.72], (0.72, 0.95], ..., (0.72, 0.95], (0.72, 0.95], (0.72, 0.95], (0.27, 0.5], (0.05, 0.27]]
Length: 20
Categories (4, interval[float64, right]): [(0.05, 0.27] < (0.27, 0.5] < (0.5, 0.72] < (0.72, 0.95]]

Opcja precision=2 ogranicza precyzję wartości zmiennoprzecinkowych do dwóch miejsc po przecinku.
Funkcja pandas.qcut działa podobnie, ale dzieli dane na kwartyle. Zwykle funkcja pandas.cut nie dzieli zbioru na koszyki o takiej samej liczbie elementów (zależy to od rozkładu danych). Funkcja pandas.qcut korzysta z kwartyli, a więc wygenerowane przez nią koszyki będą charakteryzowały się zbliżoną liczbą elementów:

In [57]:
data = np.random.standard_normal(1000)
quartiles = pd.qcut(data, 4, precision=2)

In [58]:
quartiles

[(-0.72, -0.029], (-3.15, -0.72], (0.6, 2.7], (-0.029, 0.6], (0.6, 2.7], ..., (-0.029, 0.6], (-3.15, -0.72], (-3.15, -0.72], (-3.15, -0.72], (-0.029, 0.6]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.15, -0.72] < (-0.72, -0.029] < (-0.029, 0.6] < (0.6, 2.7]]

In [59]:
quartiles.value_counts()

(-3.15, -0.72]     250
(-0.72, -0.029]    250
(-0.029, 0.6]      250
(0.6, 2.7]         250
Name: count, dtype: int64

Do funkcji pandas.cut możesz przekazać definicje własnych kwartyli (liczby z zakresu od 0 do 1 — przedział obustronnie domknięty):

In [60]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-1.371, -0.0285], (-1.371, -0.0285], (1.218, 2.7], (-0.0285, 1.218], (1.218, 2.7], ..., (-0.0285, 1.218], (-1.371, -0.0285], (-3.137, -1.371], (-3.137, -1.371], (-0.0285, 1.218]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.137, -1.371] < (-1.371, -0.0285] < (-0.0285, 1.218] < (1.218, 2.7]]


Do funkcji pandas.cut i pandas.qcut wrócimy w dalszej części rozdziału przy okazji agregacji i operacji przeprowadzanych na grupach. Wymienione funkcje dyskretyzacji są szczególnie przydatne podczas analizy kwartyli i grup.

### Wykrywanie i filtrowanie elementów odstających
Filtrowanie i przekształcanie elementów odstających polega głównie na przeprowadzaniu operacji tablicowych. Oto przykładowa ramka danych z wartościami charakteryzującymi się rozkładem normalnym:

In [61]:
data = pd.DataFrame(np.random.standard_normal((1000, 4)))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.048201,-0.005171,0.023896,0.045231
std,1.005531,0.952328,0.988716,0.970592
min,-2.947355,-3.309324,-2.771614,-3.092071
25%,-0.760096,-0.650650,-0.636381,-0.621154
50%,-0.056273,-0.035723,0.018872,0.038497
75%,0.591809,0.640089,0.703596,0.711338
max,3.050744,3.509247,4.733617,2.956932


Załóżmy, że chcemy znaleźć elementy, których wartość bezwzględna jest większa od 3:

In [62]:
col = data[2]
col[np.abs(col) > 3]

120    3.324609
456    3.248819
667    4.733617
Name: 2, dtype: float64

W celu wybrania wszystkich wierszy z wartościami przekraczającymi 3 lub –3 należy skorzystać z metody any obiektu DataFrame:


In [63]:
data[(data.abs() > 3).any(axis="columns")]

,0,1,2,3
120,-0.092863,-1.284123,3.324609,0.330652
137,-0.409640,3.509247,0.483421,-1.526763
271,3.050744,-0.127186,0.216839,0.200668
284,-0.059704,-0.671734,-1.602738,-3.092071
456,-0.289001,1.030232,3.248819,0.130982
667,1.880008,-0.349536,4.733617,1.330720
907,1.210899,-3.309324,-1.475248,-0.306219
950,-0.352215,3.007498,-1.184207,1.059201


Nawiasy obejmujące wyrażenie data.abs() > 3 są niezbędne,aby wynik porównania można było przetworzyć za pomocą metody any.
Wartości mogą być określane na podstawie tych kryteriów. Oto kod wyszukujący wartości poza zakresem od –3 do 3:

In [64]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.048252,-0.005379,0.021589,0.045323
std,1.005376,0.949531,0.980118,0.970298
min,-2.947355,-3.000000,-2.771614,-3.000000
25%,-0.760096,-0.650650,-0.636381,-0.621154
50%,-0.056273,-0.035723,0.018872,0.038497
75%,0.591809,0.640089,0.703596,0.711338
max,3.000000,3.000000,3.000000,2.956932


Instrukcja np.sign(data) generuje wartości 1 i –1 w zależności od tego, czy wartości obiektu data są dodatnie, czy ujemne:

In [65]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,1.0,1.0
1,1.0,-1.0,1.0,1.0
2,1.0,-1.0,1.0,1.0
3,-1.0,1.0,-1.0,1.0
4,1.0,-1.0,1.0,-1.0


### Permutacje i próbkowanie losowe

Permutację (czyli losową zmianę kolejności) obiektu typu Series lub wiersza ramki danych można przeprowadzić z łatwością za pomocą funkcji numpy.random.permutation. Wywołanie funkcji permutation i przekazanie jej w roli argumentu długości osi, która ma zostać poddana permutacji, powoduje wygenerowanie tablicy wartości całkowitoliczbowych określających nową kolejność:

In [66]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))

In [67]:
df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [68]:
sampler = np.random.permutation(5)
sampler

array([2, 1, 4, 3, 0])

Tablica ta może być następnie użyta w indeksowaniu opartym na funkcji iloc lub podobnej funkcji take:

In [69]:
df.take(sampler)

,0,1,2,3,4,5,6
2,14,15,16,17,18,19,20
1,7,8,9,10,11,12,13
4,28,29,30,31,32,33,34
3,21,22,23,24,25,26,27
0,0,1,2,3,4,5,6


In [70]:
df.iloc[sampler]

,0,1,2,3,4,5,6
2,14,15,16,17,18,19,20
1,7,8,9,10,11,12,13
4,28,29,30,31,32,33,34
3,21,22,23,24,25,26,27
0,0,1,2,3,4,5,6


Wywołując metodę take z argumentem axis="columns", można uzyskać wynik permutacji kolumn:

In [71]:
column_sampler = np.random.permutation(7)
column_sampler

array([2, 5, 0, 6, 4, 1, 3])

In [72]:
df.take(column_sampler, axis="columns")

,2,5,0,6,4,1,3
0,2,5,0,6,4,1,3
1,9,12,7,13,11,8,10
2,16,19,14,20,18,15,17
3,23,26,21,27,25,22,24
4,30,33,28,34,32,29,31


W celu wybrania losowego podzbioru bez zastępowania (tj. tak, aby żaden wiersz nie pojawił się dwukrotnie) można skorzystać z metody sample obiektu Series lub DataFrame:

In [73]:
df.sample(n=3)

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
1,7,8,9,10,11,12,13


W celu wygenerowania próbki z zastępowaniem (elementy mogą być wybierane więcej niż raz) skorzystaj z funkcji sample z argumentem replace=True:

In [74]:
choices = pd.Series([5, 7, -1, 6, 4])
choices.sample(n=10, replace=True)

4    4
3    6
4    4
0    5
1    7
4    4
4    4
4    4
2   -1
0    5
dtype: int64

### Przetwarzanie wskaźników i zmiennych zastępczych
Innym rodzajem transformacji przydatnej podczas modelowania statystycznego lub uczenia maszynowego jest konwersja zmiennej kategorycznej na macierz „elementów zastępczych” lub „wskaźników”. Jeżeli w kolumnie ramki danych znajduje się k odmiennych wartości, to można utworzyć na jej podstawie macierz pochodną zawierającą k kolumn wypełnionych jedynkami i zerami. Służy do tego funkcja pandas.get_dummies, ale funkcję taką można z łatwością napisać samemu. Wróćmy do zaprezentowanego wcześniej przykładu ramki danych:

In [75]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})

In [76]:
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [77]:
pd.get_dummies(df['key'])

,a,b,c
0,False,True,False
1,False,True,False
2,True,False,False
3,False,False,True
4,True,False,False
5,False,True,False


Czasami zachodzi potrzeba dodania prefiksu do nazw kolumn wygenerowanej ramki danych, które można następnie połączyć z innymi danymi. Do tego właśnie służy argument prefix funkcji pandas.get_dummies:

In [78]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,False,True,False
1,1,False,True,False
2,2,True,False,False
3,3,False,False,True
4,4,True,False,False
5,5,False,True,False


Jeżeli wiersz ramki danych należy do wielu kategorii, należy zmienne zastępcze utworzyć w inny sposób. Przyjrzyjmy się zbiorowi danych MovieLens 1M:

In [83]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('../content/movies.dat', sep='::',header=None, names=mnames, engine='python')

In [84]:
movies

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


Obiekt Series posiada specjalną metodę str.get_dummies, przystosowaną do sytuacji, w których przynależność do wielu grup jest zakodowana za pomocą ciągu ich nazw oddzielonych separatorem:


In [85]:
dummies = movies["genres"].str.get_dummies("|")
dummies.iloc[:10, :6]

,Action,Adventure,Animation,Children's,Comedy,Crime
0,0,0,1,1,1,0
1,0,1,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0
5,1,0,0,0,0,1
6,0,0,0,0,1,0
7,0,1,0,1,0,0
8,1,0,0,0,0,0
9,1,1,0,0,0,0



Teraz tak jak wcześniej możemy połączyć ten wynik z obiektem movies podczas dodawania za pomocą metody add_prefix prefiksu "Genre_" do nazw kolumn zawartych w obiekcie DataFrame:


In [86]:
movies_windic = movies.join(dummies.add_prefix('Genre_')) 
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Action                                   0
Genre_Adventure                                0
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Crime                                    0
Genre_Documentary                              0
Genre_Drama                                    0
Genre_Fantasy                                  0
Genre_Film-Noir                                0
Genre_Horror                                   0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Romance                                  0
Genre_Sci-Fi                                   0
Genre_Thriller                                 0
Genre_War                                      0
Genre_Western       

Przedstawiona metoda tworzenia zmiennych wskazujących działa dość wolno przy dużych zbiorach danych. W takich przypadkach lepiej jest napisać niskopoziomową funkcję zapisującą dane bezpośrednio w tablicy NumPy, a następnie obudować wynik jej pracy obiektem DataFrame.

In [87]:
np.random.seed(12345) # Aby przykład był powtarzalny 
values = np.random.uniform(size=10)

In [88]:
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [89]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,False,False,False,False,True
1,False,True,False,False,False
2,True,False,False,False,False
3,False,True,False,False,False
4,False,False,True,False,False
5,False,False,True,False,False
6,False,False,False,False,True
7,False,False,False,True,False
8,False,False,False,True,False
9,False,False,False,True,False


## Rozszerzone typy danych

Biblioteka pandas została pierwotnie zbudowana na bazie funkcjonalności obecnie dostępnych w NumPy — macierzowej bibliotece obliczeniowej, używanej głównie do pracy z danymi liczbowymi. Wiele z tych funkcjonalności, na przykład obsługa brakujących danych, zostało zaimplementowanych z myślą o maksymalnej kompatybilności z bibliotekami, które wykorzystują jednocześnie NumPy i pandas.

Bazowanie na bibliotece NumPy jest przyczyną wielu mankamentów biblioteki pandas, takich jak na przykład:
- Niepełna obsługa brakujących danych całkowitoliczbowych i logicznych. W efekcie biblioteka pandas używa do reprezentowania brakujących danych obiektu np.nan, a istniejące wartości przekształca w liczby zmiennoprzecinkowe, co w wielu algorytmach skutkuje subtelnymi błędami.
- Operacje wykonywane na dużych zbiorach danych tekstowych są kosztowne obliczeniowo i zajmują dużo pamięci.
- Niektóre typy danych, m.in. szeregi czasowe, interwały i znaczniki z identyfikatorami stref nie mogą być wydajnie przetwarzane bez użycia kosztownych obliczeniowo tablic obiektów Pythona.

Ostatnio biblioteka pandas została rozbudowana o rozszerzone typy danych, dzięki którym można przetwarzać nowe typy, nieobsługiwane natywnie przez bibliotekę NumPy. Wraz z tablicami można je traktować jako typy pierwszoklasowe.
Przyjrzyjmy się przykładowi utworzenia serii liczb całkowitych z brakującą wartością:

In [90]:
s = pd.Series([1, 2, 3, None])
s

0    1.0
1    2.0
2    3.0
3    NaN
dtype: float64

In [91]:
s.dtype

dtype('float64')

Głównie ze względu na kompatybilność wsteczną typ Series wykorzystuje standardowy typ float64 i obiekt np.nan oznaczający brakujące wartości. Tego rodzaju serię można również utworzyć, używając metody pandas.Int64Dtype:

In [92]:
s = pd.Series([1, 2, 3, None], dtype=pd.Int64Dtype())

In [93]:
s

0       1
1       2
2       3
3    <NA>
dtype: Int64

In [94]:
s.isna()

0    False
1    False
2    False
3     True
dtype: bool

In [95]:
s.dtype

Int64Dtype()

Ciąg <NA> oznacza brak wartości w tablicy zawierającej dane rozszerzonego typu. Jest to specjalny obiekt zastępczy pandas.NA:

In [96]:
s[3]

<NA>

In [97]:
s[3] is pd.NA

True

Zamiast metody pd.Int64Dtype można użyć krótkiego oznaczenia rozszerzonego typu "Int64". Ważna jest wielkość liter. Jeżeli zostanie użyta inna, typ zostanie potraktowany jako nierozsze- rzony, właściwy dla biblioteki NumPy:


In [98]:
s = pd.Series([1, 2, 3, None], dtype="Int64")

Biblioteka pandas zawiera również rozszerzony typ danych tekstowych, który nie wykorzystuje tablic obiektów NumPy (wymaga natomiast biblioteki pyarrow, którą trzeba zainstalować osobno):

In [99]:
s = pd.Series(['one', 'two', None, 'three'], dtype=pd.StringDtype())


In [100]:
s

0      one
1      two
2     <NA>
3    three
dtype: string

Duże tablice danych tego typu zazwyczaj zajmują znacznie mniej pamięci, a operacje na nich są bardziej wydajne.
Innym ważnym typem rozszerzonym jest Categorical. Tabela przedstawia pełną listę rozszerzonych typów aktualnie dostępnych.


![Tabela](typy.png "Tabela")

In [102]:
df = pd.DataFrame({"A": [1, 2, None, 4], "B": ["one", "two", "three", None], "C": [False, None, False, True]})

In [103]:
df

,A,B,C
0,1.0,one,False
1,2.0,two,None
2,NaN,three,False
3,4.0,None,True


In [104]:
df["A"] = df["A"].astype("Int64") 
df["B"] = df["B"].astype("string") 
df["C"] = df["C"].astype("boolean")

In [105]:
df

,A,B,C
0,1,one,False
1,2,two,<NA>
2,<NA>,three,False
3,4,<NA>,True


## Operacje przeprowadzane na łańcuchach
Popularność Pythona jako języka przeznaczonego do manipulowania nieprzetwarzanymi wcześniej danymi wynika między innymi z łatwości wykonywania operacji na łańcuchach i przetwarzania tekstu. Większość operacji tekstowych można wykonać prosto za pomocą wbudowanych metod obiektu typu string. W celu bardziej złożonych operacji dobierania wzorców i obróbki tekstu może zachodzić konieczność korzystania z wyrażeń regularnych. Pakiet pandas umożliwia stosowanie operacji łańcuchowych i wyrażeń regularnych na całych tablicach danych i dodatkowo pozwala rozwiązać problem występowania brakujących danych.

### Metody obiektu typu string

Wbudowane metody łańcuchów wystarczają do przeprowadzania wielu operacji przetwarzania łańcuchów i tworzenia skryptów. Np. metoda split dzieli na fragmenty łańcuch, w którym rolę separatora pełni przecinek:

In [106]:
val = 'a,b, guido'
val.split(',')

['a', 'b', ' guido']

In [107]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

Metoda split jest często używana w połączeniu z metodą strip, która usuwa białe znaki (w tym również znaki końca wiersza):

In [108]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

Podłańcuchy można połączyć za pomocą podwójnego dwukropka i operacji dodawania:

In [109]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

Nie jest to jednak praktyczne rozwiązanie. Można to zrobić szybciej, przekazując listę lub krotkę do metody join łańcucha '::':

In [110]:
'::'.join(pieces)

'a::b::guido'

Pozostałe metody są przeznaczone do szukania podłańcuchów. Podłańcuchy najlepiej jest wyszukiwać za pomocą słowa kluczowego in, ale można korzystać również z metod index i find:


In [111]:
'guido' in val

True

In [112]:
val.index(',')

1

In [113]:
val.find(':')

-1

Różnicą pomiędzy metodą find i index jest to, że metoda index generuje wyjątek w razie nieznalezienia łańcucha (metoda find zwraca wtedy wartość –1):

In [114]:
val.index(':')

ValueError: substring not found

Metoda count zwraca liczbę wystąpień określonego podłańcucha:

In [115]:
val.count(',')

2

Metoda replace zastępuje miejsca występowania jednego ciągu znaków innym ciągiem znaków.
Jest ona często używana do kasowania podłańcuchów (wystarczy przekazać do niej pusty łańcuch):


In [116]:
val.replace(',', '::')

'a::b:: guido'

In [117]:
val.replace(',', '')

'ab guido'

W kontekście wielu tych operacji można również stosować wyrażenia regularne.

### Wyrażenia regularne
Stosowanie wyrażeń regularnych to uniwersalny sposób wyszukiwania lub dobierania (często złożonych) ciągów znaków w tekście. Pojedyncze wyrażenie regularne określa się mianem wyrażenia regex. Jest to łańcuch utworzony zgodnie ze składnią języka wyrażeń regularnych. Wbudowany moduł Pythona re jest używany do obsługi wyrażeń regularnych w kontekście łańcuchów. 

Funkcje modułu re można podzielić na trzy kategorie: dopasowywanie ciągu, zastępowanie i dzielenie. Oczywiście wszystkie te operacje są ze sobą związane. Wyrażenie regularne opisuje wzorzec, który ma zostać znaleziony w tekście, a więc można je stosować w celu wykonania różnych operacji. Przyjrzyj się następującemu przykładowi: załóżmy, że chcemy podzielić łańcuch zawierający zmienną liczbę białych znaków (spacji, tabulacji, znaków nowego wiersza). Wyrażenie regularne \s+ jest symbolem zastępczym przynajmniej jednego białego znaku:

In [119]:
import re
text = "foo bar\tbaz \tqux"
re.split(r'\s+', text)

['foo', 'bar', 'baz', 'qux']

Po uruchomieniu funkcji re.split('\s+', text) wyrażenie regularne jest najpierw rekompilowane, a następnie na przekazanym tekście wywoływana jest metoda split. Możesz samodzielnie skompilować wyrażenie regularne za pomocą funkcji re.compile, co pozwala na uzyskanie obiektu wyrażenia regularnego, który może być użyty wielokrotnie:

In [121]:
regex = re.compile(r'\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

Gdybyś natomiast chciał uzyskać listę wszystkich wzorców pasujących do wyrażenia regularnego, możesz skorzystać z metody findall:

In [122]:
regex.findall(text) 

[' ', '\t', ' \t']

Aby uniknąć korzystania w wyrażeniach regularnych z sekwencji specjalnej rozpoczynającej się od znaku \, korzystaj z surowych literałów łańcuchowych, takich jak r'C:\x', zamiast z ich odpowiedników w postaci 'C:\\x'.
W przypadku stosowania tego samego wyrażenia do wielu łańcuchów zaleca się tworzenie obiektu wyrażenia regularnego za pomocą funkcji re.compile. Dzięki temu zmniejszona zostanie liczba użytych cykli procesora.
Metody match i search są związane z metodą findall. Metoda findall zwraca wszystkie przypadki dopasowania znalezione w łańcuchu, a metoda search zwraca tylko pierwsze dopasowanie, a dokładnie rzecz biorąc, zwraca ona początek wyszukiwanego ciągu znaków. 

Przyjrzyjmy się przykładowi wyrażenia regularnego mogącego zidentyfikować większość adresów poczty elektronicznej umieszczonych w bloku tekstu:

In [123]:
text = """Dave dave@google.com Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern =r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
# Flaga re.IGNORECASE sprawia, że wyrażenie regularne nie zwraca uwagi na wielkość liter.
regex = re.compile(pattern, flags=re.IGNORECASE)


Użycie metody findall na tekście zwraca listę adresów:

In [124]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

Metoda search zwraca specjalny obiekt dopasowania dla pierwszego adresu znalezionego w tekście. W przypadku zaprezentowanego wcześniej wyrażenia regularnego obiekt dopasowania może tylko poinformować nas o początku i końcu wzorca znalezionego w łańcuchu:

In [125]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [126]:
text[m.start():m.end()] 

'dave@google.com'

Metoda regex.match zwraca None, ponieważ jest ona w stanie znaleźć wzorzec tylko wtedy, gdy znajduje się on na początku łańcucha:

In [127]:
print(regex.match(text))
  

None


Metoda sub zwróci nowy łańcuch, w którym miejsca wystąpień wzorca zostaną zastąpione nowym łańcuchem:

In [128]:
print(regex.sub('REDACTED', text))

Dave REDACTED Steve REDACTED
Rob REDACTED
Ryan REDACTED



 Załóżmy, że chcesz znaleźć adres e-mail i jednocześnie dokonać segmentacji każdego adresu na trzy komponenty: nazwę użytkownika, nazwę domeny i sufiks domeny. W tym celu musisz umieścić nawiasy okrągłe wokół części wzorca, które mają być poddane segmentacji:

In [129]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})' 
regex = re.compile(pattern, flags=re.IGNORECASE)

Obiekt dopasowania wygenerowany przez to zmodyfikowane wyrażenie regularne zwraca krotkę z komponentami wzorca (w celu uzyskania do niej dostępu należy skorzystać z metody groups):

In [130]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

Gdy wzorzec zawiera grupy, metoda findall zwraca listę krotek:

In [131]:
regex.findall(text) 

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

Metoda sub umożliwia uzyskanie dostępu do grup każdego dopasowania za pomocą symboli w rodzaju \1 i \2. Symbol \1 odwołuje się do pierwszej dopasowanej grupy, symbol \2 odwołuje się do drugiej dopasowanej grupy itd.:


In [132]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



Python obsługuje o wiele więcej wyrażeń regularnych, ale korzystanie z większości z nich wykracza poza zakres tematyczny tej książki. W tabeli podsumowano wybrane metody wyrażeń regularnych.

![Tabela](regexp.png "Tabela")

### Funkcje tekstowe w pakiecie pandas
 Oczyszczanie zbioru danych przed analizą często wiąże się z koniecznością przekształcania łańcuchów. Sprawę komplikują dodatkowo brakujące dane kolumny z łańcuchami:

In [133]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com', 'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)

In [134]:
data.isna ()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

W celu zastosowania metod łańcucha lub wyrażenia regularnego na każdej wartości (przekazując funkcję lambda lub inną funkcję), można skorzystać z notacji data.map, ale w przypadku wartości NA (null) wywoła to komunikat błędu. W związku z tym obiekt Series dysponuje metodami tablicowymi przeznaczonymi do przetwarzania łańcuchów, które pomijają brakujące wartości. Dostęp do tych metod uzyskuje się za pomocą atrybutu str obiektu typu Series. Za pomocą metody str.contains możemy np. sprawdzić, czy każdy adres e-mail zawiera ciąg znaków 'gmail':

In [135]:
data.str.contains('gmail') 

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Zwróć uwagę, że wynikiem tej operacji jest obiekt. Biblioteka pandas implementuje rozszerzone typy danych, umożliwiające wykonywanie na ciągach znaków, liczbach całkowitych i wartościach logicznych specjalnych operacji, z którymi do niedawna były pewne problemy, jeżeli brakowało wartości:

In [136]:
data_as_string_ext = data.astype('string')

In [137]:
data_as_string_ext


Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                 <NA>
dtype: string

In [138]:
data_as_string_ext.str.contains("gmail") 

Dave     False
Steve     True
Rob       True
Wes       <NA>
dtype: boolean

Możliwe jest również zastosowanie wyrażeń regularnych oraz innych opcji modułu re, takich jak IGNORECASE:

In [139]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [140]:
data.str.findall(pattern, flags=re.IGNORECASE) 

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

Istnieje kilka sposobów przeprowadzenia wektorowego wyszukiwania elementów. Można to zrobić za pomocą funkcji str.get lub poprzez przekazanie indeksu do funkcji str:

In [141]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0] 
matches



Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [142]:
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

Łańcuch możesz również pociąć za pomocą następującej składni:

In [143]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

Metoda str.extract zwraca obiekty DataFrame zawierające grupy wyodrębnione za pomocą wyrażenia regularnego:

In [144]:
data.str.extract(pattern, flags=re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


Więcej metod obiektów typu string 
![Tabela](str1.png "Tabela")
![Tabela](str2.png "Tabela")


## Dane kategoryczne
W tej sekcji wprowadzę typ Categorical obsługiwany przez pakiet pandas. Dowiesz się, jak można z niego korzystać w celu uzyskania lepszej wydajności i zmniejszenia zużycia pamięci podczas wykonywania niektórych operacji. Ponadto przedstawię wybrane narzędzia przeznaczone do używania danych kategorycznych w zastosowaniach związanych ze statystyką i uczeniem maszynowym.

### Kontekst i motywacja

Często kolumna tabeli zawiera powtarzające się instancje mniejszego zbioru różnych wartości. Funkcje unique i calue_counts, które wyciągają odmienne wartości z tablicy i określają częstotliwości ich występowania:

In [145]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [146]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [148]:
pd.Series.value_counts(values)

apple     6
orange    2
Name: count, dtype: int64

Wiele systemów obsługujących dane (składujących je, przetwarzających w celach statystycznych itd.) korzysta ze specjalnych technik przedstawiania danych zawierających powtarzające się wartości, co pozwala na zmniejszenie objętości danych i przyśpieszanie ich przetwarzania. Najlepszą praktyką stosowaną w przypadku hurtowni danych jest korzystanie z tablic wymiaru zawierających unikalne wartości. W takim przypadku pierwotne obserwacje są zapisywane w postaci kluczy numerycznych odwołujących się do tabeli wymiaru:

In [149]:
values = pd.Series([0, 1, 0, 0] * 2)

In [150]:
dim = pd.Series(['apple', 'orange'])


In [151]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [152]:
dim

0     apple
1    orange
dtype: object

W celu przywrócenia początkowej struktury serii łańcuchów możemy skorzystać z metody take:

In [153]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

Przedstawienie danych za pomocą wartości liczbowych określamy mianem reprezentacji kategorycznej lub kodowanej słownikowo. Tablicę unikalnych wartości określamy mianem kategorii, słownika lub poziomów danych. W tej książce będę posługiwał się terminami reprezentacja kategoryczna i kategorie. Wartości liczbowe odwołujące się do kategorii określamy mianem kodów kategorii lub po prostu kodów.
Reprezentacja kategoryczna pozwala na znaczne zwiększenie wydajności przeprowadzania procesów analitycznych. Ponadto umożliwia przeprowadzanie transformacji kategorii bez modyfikowania kodów. Transformacje, które można przeprowadzić względnie niskim kosztem, to:
- zmiana nazw kategorii;
- dodanie nowej kategorii bez zmieniania kolejności i położenia utworzonych wcześniej kategorii.

### Rozszerzony typ Categorical w bibliotece pandas
Biblioteka pandas oferuje specjalny, rozszerzony typ danych Categorical przeznaczony do przechowywania danych, w których kategorie są przedstawiane (zakodowane) przy użyciu wartości liczbowych. Jest to popularna technika kompresji danych, w których wielokrotnie występują podobne wartości. Zapewnia znacznie wyższą wydajność przy mniejszym zużyciu pamięci, zwłaszcza w przypadku danych łańcuchowych. Przyjrzyjmy się jeszcze raz użytemu wcześniej obiektowi typu Series:


In [154]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2 
N = len(fruits)
rng = np.random.default_rng(seed=12345)
df = pd.DataFrame({'fruit': fruits, 
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N), 
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])

In [155]:
df

,basket_id,fruit,count,weight
0,0,apple,3,3.229155
1,1,orange,5,2.508377
2,2,apple,12,3.631700
3,3,apple,14,2.225589
4,4,apple,13,3.359677
5,5,orange,4,0.201952
6,6,apple,5,3.224939
7,7,apple,9,3.723263


W zaprezentowanym przykładzie df['fruit'] jest tablicą obiektów typu string (łańcuchów). Możemy dokonać jej konwersji na typ kategoryczny za pomocą następującej funkcji:

In [156]:
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

Teraz wartościami parametru fruit_cat są instancje typu pandas.Categorical, do których można się odwoływać za pomocą atrybutu array:


In [157]:
c = fruit_cat.array
type(c)

pandas.core.arrays.categorical.Categorical

Obiekt Categorical ma atrybuty categories (kategorie) i codes (kody):

In [158]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [159]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

Do tych danych można się łatwiej odwoływać za pomocą atrybutu dostępowego cat, który będzie wkrótce opisany.
Powiązania pomiędzy kodami a kategoriami można uzyskać, stosując następującą przydatną sztuczkę:

In [160]:
dict(enumerate(c.categories)) 

{0: 'apple', 1: 'orange'}

Możliwe jest wykonanie konwersji kolumny obiektu DataFrame na obiekt typu Categorical poprzez operację przypisania:

In [161]:
df['fruit'] = df['fruit'].astype('category')
df['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

Instancje obiektów pandas.Categorical mogą być tworzone bezpośrednio na podstawie innych typów sekwencji Pythona:

In [162]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])

my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

W przypadku uzyskania danych zakodowanych kategorycznie z innego źródła możesz skorzystać z alternatywnego konstruktora from_codes:

In [163]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2


['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

Jeśli kolejność kategorii nie zostanie określona w sposób jawny, to podczas konwersji nie przyjmuje się żadnego konkretnego sposobu sortowania kategorii. W związku z tym kolejność elementów tablicy categories zależy od kolejności danych wejściowych. Podczas korzystania z from_codes lub innego konstruktora możesz określić konieczność ustawienia kategorii w sposób uporządkowany:

In [164]:
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cat


['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

Wygenerowany komunikat [foo < bar < baz] informuje o tym, że element 'foo' znajduje się przed elementem 'bar' itd. Kolejność instancji obiektu typu Categorical można określić później za pomocą metody as_ordered:

In [165]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

Dane kategoryczne nie muszą być łańcuchami (mimo że we wszystkich przytoczonych przeze mnie przykładach są one właśnie łańcuchami). Tablica kategoryczna może zawierać dowolne wartości typu niemodyfikowalnego.

### Obliczenia na obiektach typu Categorical
Ogólnie rzecz biorąc, obiekty typu Categorical i ich odpowiedniki pozbawione kodowania kategorii działają tak samo. Niektóre funkcje pakietu pandas działają sprawniej z obiektami kategorycznymi (dotyczy do np. funkcji groupby). Niektóre funkcje mogą również korzystać z flagi ordered.
Przyjrzyjmy się przykładowi losowych danych numerycznych przetwarzanych za pomocą funkcji tworzącej koszyki danych — pandas.qcut. W wyniku zaprezentowanej operacji zwrócony zostanie obiekt pandas.Categorical. 


In [166]:
rng = np.random.default_rng(seed=12345)
draws = rng.standard_normal(1000)
draws[:5]

array([-1.42382504,  1.26372846, -0.87066174, -0.25917323, -0.07534331])

Podzielmy dane na koszyki określające kwartyle i obliczmy kilka parametrów statystycznych:

In [167]:
bins = pd.qcut(draws, 4)
bins

[(-3.121, -0.675], (0.687, 3.211], (-3.121, -0.675], (-0.675, 0.0134], (-0.675, 0.0134], ..., (0.0134, 0.687], (0.0134, 0.687], (-0.675, 0.0134], (0.0134, 0.687], (-0.675, 0.0134]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.121, -0.675] < (-0.675, 0.0134] < (0.0134, 0.687] < (0.687, 3.211]]

Dokładne granice kwartyli są przydatne, ale w raportach lepiej jest używać ich nazw. Można w tym celu użyć argumentu labels metody qcut:

In [168]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins
bins.codes[:10]

array([0, 3, 0, 1, 1, 0, 0, 2, 2, 0], dtype=int8)

Oznaczone etykietami kategorie bins nie zawierają informacji o krawędziach koszyków danych, więc w celu obliczenia parametrów statystycznych skorzystamy z funkcji groupby:

In [169]:
bins = pd.Series(bins, name='quartile')

In [170]:
results = (pd.Series(draws).groupby(bins, observed=True)
.agg(['count', 'min', 'max']) .reset_index())
results

,quartile,count,min,max
0,Q1,250,-3.119609,-0.678494
1,Q2,250,-0.673305,0.008009
2,Q3,250,0.018753,0.686183
3,Q4,250,0.688282,3.211418


Kolumna 'quartile' obiektu wyjściowego zawiera informacje o kategoriach koszyków, w tym między innymi określa ich kolejność:

In [171]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

#### Obiekty kategoryczne a zwiększenie wydajności

Typy kategoryczne poprawiają wydajność operacji i zajmują mniej pamięci. Przeanalizujmy więc kilka przykładów. Przyjrzyjmy się obiektowi Series zawierającemu 10 milionów elementów należących do małej liczby unikalnych kategorii:

In [172]:
N = 10_000_000
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

 Teraz dokonam konwersji etykiet (labels) na dane kategoryczne:

In [173]:
categories = labels.astype('category')

Obiekt labels zajmuje o wiele więcej miejsca w pamięci od obiektu categories:

In [174]:
labels.memory_usage(deep=True)

520000132

In [175]:
categories.memory_usage(deep=True) 

10000512

Konwersja na dane kategoryczne wymaga również pewnych kosztów, ale są one ponoszone jednorazowo:

In [176]:
%time _ = labels.astype('category')

CPU times: user 156 ms, sys: 27.8 ms, total: 183 ms
Wall time: 190 ms


Operacje grupowania mogą zostać znacznie przyśpieszone w wyniku zastosowania danych kategorycznych, ponieważ algorytm tego typu operacji korzysta z tablic zawierających kody w postaci liczb zastępujących łańcuchy znaków. Porównajmy wydajność metody value_counts, która wewnętrznie korzysta z algorytmu funkcji groupby:

In [177]:
%timeit labels.value_counts()

158 ms ± 974 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [178]:
%timeit categories.value_counts()

16.7 ms ± 327 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Metody obiektu kategorycznego
Obiekt typu Series zawierający dane kategoryczne dysponuje kilkoma specjalnymi metodami podobnymi do specjalnych metod Series.str przeznaczonych bo obsługi łańcuchów znaków. Metody te umożliwiają również wygodne uzyskanie dostępu do kategorii i kodów. Przyjrzyj się następującemu przykładowi obiektu typu Series:


In [179]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

Specjalny atrybut cat umożliwia uzyskanie dostępu do metod kategorycznych:

In [180]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [181]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

Załóżmy, że wiemy o tym, że rzeczywisty zbiór kategorii tych danych wykracza poza cztery zaobserwowane wartości. W celu zmodyfikowania zbioru kategorii możemy skorzystać z metody set_categories:


In [182]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

Dane wydają się niezmodyfikowane, ale nowe kategorie zostaną odzwierciedlone w korzystających z nich operacjach. Funkcja value_counts jest przykładem funkcji biorącej pod uwagę obecne kategorie:

In [183]:
cat_s.value_counts()

a    2
b    2
c    2
d    2
Name: count, dtype: int64

In [184]:
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
Name: count, dtype: int64

W przypadku dużych zbiorów danych konwersja na dane kategoryczne może być wygodnym narzędziem pozwalającym na zmniejszenie zapotrzebowania na pamięć i przyśpieszenie przetwarzania danych. Po przefiltrowaniu dużego obiektu typu DataFrame lub Series wiele kategorii może nie wystąpić w danych. W celu rozwiązania tego problemu poprzez usunięcie kategorii, do których nie należy żadna obserwacja, możemy skorzystać z metody remove_unused_categories:


In [185]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]

In [186]:
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [187]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

Listę obsługiwanych metod kategorycznych znajdziesz w tabeli
![Tabela](kat.png "Tabela")


### Tworzenie fikcyjnych zmiennych używanych podczas modelowania
Podczas korzystania z narzędzi statystycznych i przeznaczonych do uczenia maszynowego bardzo często przeprowadza się konwersję danych kategorycznych na zmienne fikcyjne (ang. dummy variable) — jest to tzw. kodowanie one hot. Wymaga ono utworzenia ramki danych zawierającej kolumny dla wszystkich unikalnych kategorii. Kolumny te są wypełniane wartością 1 w przypadku wystąpienia danej kategorii, a wartością 0, gdy dana kategoria nie występuje.
Przyjrzyjmy się jeszcze raz temu przykładowi:

In [188]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

Funkcja pandas.get_dummies dokonuje konwersji jednowymiarowych danych kategorycznych na obiekt DataFrame zawierający zmienną fikcyjną:

In [189]:
pd.get_dummies(cat_s)


,a,b,c,d
0,True,False,False,False
1,False,True,False,False
2,False,False,True,False
3,False,False,False,True
4,True,False,False,False
5,False,True,False,False
6,False,False,True,False
7,False,False,False,True
